In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '..') # in order to be able to import from scripts.py
from scripts.track import get_infected_img, get_events_df
from scripts.track import get_tracks, plot_tracks
from tqdm import tqdm
from pathlib import Path
import matplotlib
from scripts.client import VisAVisClient
from scripts.make_protocol import make_protocol

In [2]:
PARAMETERS_DEFAULT = {
  "c_rate": 1,
  "e_incr": 1,
  "i_incr": 1,
  "r_incr": 0.0667
}

# Tracking simulations

In [3]:
from laptrack import LapTrack

def get_laptrack(max_distance = 25):
    lt = LapTrack(
        track_dist_metric="sqeuclidean",  
        splitting_dist_metric="sqeuclidean",
        merging_dist_metric="sqeuclidean",
        track_cost_cutoff=max_distance**2,
        splitting_cost_cutoff=max_distance**2,  # False for non-splitting case
        merging_cost_cutoff=False,  # False for non-merging case
        )
    return lt

def plot_trees(track_df):
    colors = [list(np.random.choice(range(256), size=3)) for _ in range(len(track_df['tree_id'].unique()))]
    cmap = matplotlib.colors.ListedColormap(colors)
    plt.figure(figsize=(20, 10))
    plt.scatter(track_df.seconds, track_df.h, c=track_df.tree_id, cmap=cmap)

def cut_small_trees(track_df, size = 10):
    tree_count = track_df.groupby('tree_id').size()
    tree_end = track_df.groupby('tree_id').h.max()
    false_trees = tree_count[(tree_count < size) & (tree_end > 20)].index.values
    track_df = track_df[~track_df.tree_id.isin(false_trees)]
    for i, index in enumerate(false_trees):
        index -= i
        track_df.loc[track_df.tree_id > index, 'tree_id'] -= 1
    return track_df

def analize_tracks(sim_path, max_distance, roll = 20, std_factor = 1/10):
    df = pd.read_csv(sim_path)
    img = get_infected_img(df)
    lt = get_laptrack(max_distance=max_distance)
    track_df, split_df, merge_df = get_tracks(img, lt, roll=roll, std_factor=std_factor)
    track_df = cut_small_trees(track_df) 
    return track_df, split_df, merge_df

In [4]:

def debug_track(folder, max_distance, roll, std_factor, sim_id, color_by = 'tree_id'):
    sim_path = f"{folder}/out-{sim_id}.csv"
    track_df, split_df, merge_df = analize_tracks(sim_path, max_distance=max_distance, roll=roll, std_factor=std_factor)
    plot_trees(track_df)#, split_df, merge_df, color_by=color_by)
    plt.title(f"Number of trees {len(track_df.tree_id.unique())}")
    plt.show()
    return track_df,split_df,merge_df

def debug_gen(list_id, folder, max_distance=30, roll=20, std_factor=1/10):
    for i, sim_id in enumerate(list_id):
        track_df, split_df, merge_df = debug_track(folder, max_distance, roll, std_factor, sim_id)
        print(i)
        yield track_df, split_df, merge_df

# 50 Pulses 

In [5]:
roll = 20
std_factor = 1/10
max_distance = 14
sim_num = 50
folder = "results/P50-100"
pulses = 50
results = []
many_splits = []
few_splits = []

In [ ]:
for sim_id in tqdm(range(sim_num)):
    #plt.figure(figsize=(20, 10))
    sim_path = f"results/P50/out-{sim_id}.csv"
    track_df, split_df, merge_df = analize_tracks(sim_path, max_distance=30, roll=20, std_factor=1/10)
    if (track_df['tree_id'] > pulses).any():
        many_splits.append(sim_id)
    if len(track_df.tree_id.unique()) < pulses:
        few_splits.append(sim_id)
    continue
    if track_df['tree_id'].max() == 49:
        results.append((track_df, sim_id))
        plot_trees(track_df)
        plt.title(f"Trees: {len(track_df['tree_id'].unique())}")
        plt.show()

In [ ]:
deb_many = debug_gen(many_splits, folder)
deb_few = debug_gen(few_splits, folder, max_distance=12)

In [ ]:
track_df, split_df, merge_df = next(deb_few)

In [ ]:
for result, id in results:
    result.to_csv(f"results/Final/data-{id}.csv")
    activations = track_df.loc[track_df[track_df['h'] > 197.5].track_id.drop_duplicates().index]
    activations = activations.reset_index()[['seconds', 'tree_id']]
    activations = activations.reset_index().rename(columns = {'index': 'output_id', 'tree_id': 'input_id'})
    activations['seconds'] *= 4
    activations.to_csv(f"results/Activations//act-{id}.csv")

# 10 Pulses

In [ ]:
roll = 20
std_factor = 1/10
max_distance = 30
sim_num = 100
folder = "results/P10"
pulses = 10

In [ ]:
many_splits = []
few_splits = []
lt = get_laptrack(max_distance=max_distance)

for sim_id in tqdm(range(sim_num)):
    sim_path = f"results/P10/out-{sim_id}.csv"
    track_df, split_df, merge_df = analize_tracks(sim_path, max_distance=30, roll=20, std_factor=1/10)    
    if (track_df['tree_id'] > pulses).any():
        many_splits.append(sim_id)
    if len(track_df.tree_id.unique()) < pulses:
        few_splits.append(sim_id)

In [ ]:
many_splits, few_splits #= ([], [29, 43, 59, 61, 70, 71, 79, 82, 92, 97])

In [ ]:
deb_many = debug_gen(many_splits, folder)
deb_few = debug_gen(few_splits, folder, max_distance=30)

In [ ]:
track_df, split_df, merge_df = next(deb_many)

# Get data

In [ ]:
#---------      Get 10 pulses data      ----------
#pulses = 10
#sim_num = 100
#folder = "results/P10"

#---------      Get 50 pulses data      ----------
pulses = 50
sim_num = 10
folder = "results/P50-100"
Path(folder).mkdir(exist_ok=True, parents=True)

In [ ]:
for channel_height in [7]:

    client = VisAVisClient(
    )

    for interval in [100]:

        protocol_file_path = make_protocol(
            pulse_intervals = (pulses - 1 ) *  [interval] + [1500],
            duration=4,
            out_folder='./'
        )
        
        for sim in tqdm(range(sim_num)):
            result = client.run(
                parameters_json=PARAMETERS_DEFAULT,
                protocol_file_path= protocol_file_path,
            )
            result.states.to_csv(f"{folder}/out-{sim}.csv", index=False)